### Google Colab File Upload & Install Library

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving click.MOV to click.MOV
User uploaded file "click.MOV" with length 33470015 bytes


In [ ]:
!pip install mediapipe

### 라이브러리 import 및 미디어파이프 설정

In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
import time

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

### 검지손가락 각도 정보 추출하여 데이터셋 생성

#### CLICK

In [ ]:
cap = cv2.VideoCapture('click.MOV') # video open

data_click = pd.DataFrame()
while cap.isOpened():
    ret, img = cap.read()
    
    if img is None:
        break

    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    index_point = [[8, 7, 6], [7, 6, 5], [6, 5, 0]]
    point_info = np.zeros((21, 2))
    if result.multi_hand_landmarks is not None:
    for res in result.multi_hand_landmarks:

    for j, lm in enumerate(res.landmark):
        point_info[j] = [lm.x, lm.y]

    angles = []
    for point in index_point:
        # 검지손가락 포인트 각도 게산
        ba = point_info[point[0]] - point_info[point[1]]
        bc = point_info[point[2]] - point_info[point[1]]
        cosine_angle = np.dot(ba,bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cosine_angle) 
        pAngle = np.degrees(angle)
        angles.append(pAngle)

    # 라벨 추가
    angles.append(1) # 1 - click

# 데이터셋 추가
data_click = data_click.append([angles])

In [ ]:
data_click.head()

,0,1,2,3
0,22.598724,179.023686,163.249317,1
0,2.973941,179.892002,165.433054,1
0,147.763771,177.033637,165.819879,1
0,167.020688,177.214987,166.358002,1
0,154.102063,177.232321,165.434704,1


In [ ]:
data_click.tail()

,0,1,2,3
0,24.790025,167.686864,177.008100,1
0,21.537427,168.296129,176.435055,1
0,26.206861,162.897668,176.129663,1
0,28.619377,162.772284,175.529611,1
0,21.857349,166.397656,176.344917,1


In [ ]:
data_click.to_csv("dataset_click.csv") # csv 파일로 저장

#### NONE

In [ ]:
cap = cv2.VideoCapture('none.MOV') # video open

data_none = pd.DataFrame()
while cap.isOpened():
    ret, img = cap.read()
    
    if img is None:
        break

    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    index_point = [[8, 7, 6], [7, 6, 5], [6, 5, 0]]
    point_info = np.zeros((21, 2))
    if result.multi_hand_landmarks is not None:
    for res in result.multi_hand_landmarks:

    for j, lm in enumerate(res.landmark):
        point_info[j] = [lm.x, lm.y]

    angles = []
    for point in index_point:
        # 검지손가락 포인트 각도 게산
        ba = point_info[point[0]] - point_info[point[1]]
        bc = point_info[point[2]] - point_info[point[1]]
        cosine_angle = np.dot(ba,bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cosine_angle) 
        pAngle = np.degrees(angle)
        angles.append(pAngle)

    # 라벨 추가
    angles.append(0) # 0 - none

# 데이터셋 추가
data_none = data_none.append([angles])

In [ ]:
data_none.head()

,0,1,2,3
0,177.023114,176.338735,166.104406,0
0,177.680023,177.702565,167.582472,0
0,178.238596,177.121924,167.263819,0
0,178.061959,177.010129,167.606877,0
0,178.331293,176.677957,167.512249,0


In [ ]:
data_none.tail()

,0,1,2,3
0,176.423832,173.997588,171.909546,0
0,176.408986,174.194977,171.699901,0
0,176.588753,173.374272,172.118063,0
0,176.571527,173.746418,172.134432,0
0,176.515446,173.417380,172.506359,0


In [ ]:
data_none.to_csv("dataset_none.csv") # csv 파일로 저장